In [1]:
from packages import wrappers

In [2]:
wrappers.get_df_from_cryptowatch()

,Open,High,Low,Close,Volume,QuoteVolume
Date,,,,,,
2015-01-27,275.609985,276.059998,269.600006,274.480011,199.258667,0.000000e+00
2015-01-28,274.730011,278.470001,244.029999,263.649994,1609.966187,0.000000e+00
2015-01-29,263.959991,265.989990,228.500000,236.089996,5100.223633,0.000000e+00
2015-01-30,235.889999,242.000000,221.139999,235.029999,6627.899902,0.000000e+00
2015-01-31,235.000000,244.589996,227.889999,229.070007,5743.572266,0.000000e+00
...,...,...,...,...,...,...
2021-08-10,43829.150000,46497.420000,42821.300000,46285.480000,19464.724935,8.749332e+08
2021-08-11,46280.000000,46718.440000,44650.270000,45595.660000,13756.500669,6.266719e+08
2021-08-12,45601.820000,46781.090000,45350.740000,45553.490000,11628.320440,5.366116e+08
